# Prediction with classifier

In [1]:
%matplotlib inline
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
import pickle

import matplotlib.pylab as pl
import xgboost
from xgboost import plot_tree

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing
from pandas_ml import ConfusionMatrix


import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

/home/ramon/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ramon/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Given a df with demographic, economics features, predict the winner of the presidential vote for each french city.


### read data

In [2]:
df = pd.read_csv('data/DOI/elections_2017_First_round.csv')

/home/ramon/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,5,86,171) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### filter columns

In [3]:
col_target = ['best_candidate']
col_features = [ 'CODE_REG',  'POPULATION', 'LE.PEN_2012', 'HOLLANDE_2012', 'SARKOZY_2012',
 'MELENCHON_2012',  'NoJob_all_ratio', 'NoJob_1524_ratio', 'NoJob_2554_ratio',
 'NoJob_5564_ratio',
 'Job_AGRO_ratio',
 'Job_ARTISAN_ratio',
 'Job_CADRE_ratio',
 'Job_INTERMED_ratio',
 'Job_EMPLOYE_ratio',
 'Job_OUVRIER_ratio',
 'Etudiants_ratio',
 'Retraites_ratio',
 'Q213',
 'Q3_Q1',
 'RD',
 'GI13',
 'Q213_ARR',
 'Q3_Q1_ARR',
 'RD_ARR',
 'GI13_ARR',
 
 'ratio_foreigners',
 'ratio_immigrants',
 'ratio_immigrants_AGRO',
 'ratio_immigrants_ARTISANS',
 'ratio_immigrants_CADRE',
 'ratio_immigrants_INTER',
 'ratio_immigrants_EMPLOYE',
 'ratio_immigrants_OUVRIER',
 'ratio_immigrants_RETRAITE',
 'education_no_diplom_ratio',
 'education_CAPBEP_ratio',
 'education_BAC_ratio',
 'education_SUP_ratio',
 'family_monoparental_ratio',
 
 'Indice.Fiscal.Partiel',
 'Score.Fiscal',
 'Indice.Evasion.Client',
 'Score.Evasion.Client',
 'Indice.Synergie.M\xc3\xa9dicale',
 'Score.Synergie.M\xc3\xa9dicale',

 'DEP',
 
 'Densit\xc3\xa9.M\xc3\xa9dicale.BV',
 'Score.\xc3\xa9quipement.de.sant\xc3\xa9.BV',
 'Indice.D\xc3\xa9mographique',
 'Score.D\xc3\xa9mographique',
 'Indice.M\xc3\xa9nages',
 'Score.M\xc3\xa9nages',
 
 'Evolution.Population',
 'Evolution.Pop..',
 
 'Moyenne.Revenus.Fiscaux.D\xc3\xa9partementaux',
 'Moyenne.Revenus.Fiscaux.R\xc3\xa9gionaux',
 'Dep.Moyenne.Salaires.Horaires',
 'Dep.Moyenne.Salaires.Cadre.Horaires',
 'Dep.Moyenne.Salaires.Prof.Interm\xc3\xa9diaire.Horaires',
 'Dep.Moyenne.Salaires.Employ\xc3\xa9.Horaires',
 'Dep.Moyenne.Salaires.Ouvri\xc3\xa9.Horaires',
 'Reg.Moyenne.Salaires.Horaires',
 'Reg.Moyenne.Salaires.Cadre.Horaires',
 'Reg.Moyenne.Salaires.Prof.Interm\xc3\xa9diaire.Horaires',
 'Reg.Moyenne.Salaires.Employ\xc3\xa9.Horaires',
 'Reg.Moyenne.Salaires.Ouvri\xc3\xa9.Horaires',
 'Valeur.ajout\xc3\xa9e.r\xc3\xa9gionale',
 
 'Score.Urbanit\xc3\xa9',
 
 
 'Taux.\xc3\xa9tudiants',
 'Taux.Propri\xc3\xa9t\xc3\xa9',
 'Dynamique.D\xc3\xa9mographique.INSEE',
 'Capacit\xc3\xa9.Fisc',
 'Capacit\xc3\xa9.Fiscale',
 'Moyenne.Revnus.fiscaux',
 'Taux.Evasion.Client',
 
 'PIB.R\xc3\xa9gionnal',
 'Score.Croissance.Population',
 'Score.Croissance.Entrepreneuriale',
 'Score.VA.R\xc3\xa9gion',
 'Score.PIB',
 
 'Nb.Pharmacies.et.parfumerie_ratio',
 'Nb.Omnipraticiens.BV_ratio',
 'Nb.Infirmiers.Lib\xc3\xa9raux.BV_ratio',
 'Nb.dentistes.Lib\xc3\xa9raux.BV_ratio',
 'Nb.pharmaciens.Lib\xc3\xa9raux.BV_ratio',
 'Nb.M\xc3\xa9nages_ratio',
 'Nb.R\xc3\xa9sidences.Principales_ratio',
 'Nb.propri\xc3\xa9taire_ratio',
 'Nb.Logement_ratio',
 'Nb.R\xc3\xa9sidences.Secondaires_ratio',
 'Nb.Log.Vacants_ratio',
 'Nb.Occupants.R\xc3\xa9sidence.Principale_ratio',
 'Nb.Femme_ratio',
 'Nb.Homme_ratio',
 'Nb.Mineurs_ratio',
 'Nb.Majeurs_ratio',
 'Nb.Etudiants_ratio',
 'Nb.Entreprises.Secteur.Services_ratio',
 'Nb.Entreprises.Secteur.Commerce_ratio',
 'Nb.Entreprises.Secteur.Construction_ratio',
 'Nb.Entreprises.Secteur.Industrie_ratio',
 'Nb.Cr\xc3\xa9ation.Enteprises_ratio',
 'Nb.Cr\xc3\xa9ation.Industrielles_ratio',
 'Nb.Cr\xc3\xa9ation.Construction_ratio',
 'Nb.Cr\xc3\xa9ation.Commerces_ratio',
 'Nb.Cr\xc3\xa9ation.Services_ratio',
 'Nb.Atifs_ratio',
 'Nb.Actifs.Salari\xc3\xa9s_ratio',
 'Nb.Actifs.Non.Salari\xc3\xa9s_ratio',
 'Nb.Logement.Secondaire.et.Occasionnel_ratio',
 'Nb.Hotel_ratio',
 'Nb.Camping_ratio',
 'Nb.Education..sant\xc3\xa9..action.sociale_ratio',
 'Nb.Services.personnels.et.domestiques_ratio',
 'Nb.Sant\xc3\xa9..action.sociale_ratio',
                
 'Nb.Industries.des.biens.interm\xc3\xa9diaires_ratio',
 'Nb.de.Commerce_ratio',
 'Nb.de.Services.aux.particuliers_ratio',
 'Nb.institution.de.Education..sant\xc3\xa9..action.sociale..administration_ratio']
 

### Convert Corsica DEPT to numeric :

In [4]:
df.DEP = df.DEP.replace('2A',200)
df.DEP = df.DEP.replace('2B',201)

In [5]:
df.DEP = df.DEP.fillna(0)
df.DEP = pd.to_numeric(df.DEP)

In [6]:
#df.columns.to_series().groupby(df.dtypes).groups

In [7]:
df.best_candidate.unique().size

9

In [8]:
clf = xgboost.XGBClassifier()


### Train test split

In [9]:
le = preprocessing.LabelEncoder()

X = df[col_features].values
y = df[col_target].values

X_train, X_test, y_train_text,y_test_text = train_test_split( X, y, test_size=0.33, random_state=42)
y_train,y_test = le.fit_transform(y_train_text), le.fit_transform(y_test_text)


/home/ramon/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ramon/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [16]:
def cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=5):
    gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=-1,verbose=1)
    gs.fit(Xtrain, ytrain)
    pickle.dump(gs.best_estimator_, open('best_xgb_model.p', "wb"))
    return gs.best_estimator_    

In [17]:
params = {'nthread':[-1],
     'max_depth':range(3,10,2),
     'objective':['multi:softmax'],
     'learning_rate' :[0.1,0.01,0.001],
    'min_child_weight':[6,8,10,12],
    'gamma':[i/10.0 for i in range(0,5)],
     'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
          'silent':[False]
}

In [ ]:
best_clf = cv_optimize(clf,params,X_train,y_train)


Fitting 5 folds for each of 19200 candidates, totalling 96000 fits


In [13]:
best_clf = pickle.load(open('best_xgb_model.p', "rb"))

y_pred = best_clf.predict(X_test)

accuracy_score(y_test, y_pred, normalize=False)
print(classification_report(y_test, y_pred))
df_features_weight = pd.DataFrame({"features":col_features, 'importance':best_clf.feature_importances_})
df_prediction = pd.DataFrame({'best_candidate':y_test_text.ravel(),'predicted':le.inverse_transform(y_pred)})

confusion_matrix_int = ConfusionMatrix(y_test.ravel(), y_pred)
confusion_matrix_text = ConfusionMatrix(y_test_text.ravel(), le.inverse_transform(y_pred))
print("Confusion matrix:\n%s" % confusion_matrix)

             precision    recall  f1-score   support

          0       0.50      0.05      0.09       163
          1       0.00      0.00      0.00         1
          2       0.00      0.00      0.00         5
          3       0.67      0.61      0.64      1892
          4       0.00      0.00      0.00         5
          5       0.25      0.10      0.14        20
          6       0.82      0.89      0.85      6174
          7       0.64      0.64      0.64      2362
          8       0.65      0.56      0.60      1192

avg / total       0.74      0.75      0.74     11814

Confusion matrix:
<function confusion_matrix at 0x7f55603d68c0>


/home/ramon/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
confusion_matrix_int.print_stats()

Confusion Matrix:

Predicted   0  1  2     3  4  5     6     7     8  __all__
Actual                                                    
0           8  0  0    30  0  0    68    40    17      163
1           0  0  0     0  0  0     0     0     1        1
2           0  0  0     1  0  0     3     1     0        5
3           2  0  0  1162  0  3   479   223    23     1892
4           0  0  0     0  0  0     1     2     2        5
5           0  0  0     4  0  2     3    11     0       20
6           3  0  0   295  1  0  5481   282   112     6174
7           3  0  0   212  0  3   428  1505   211     2362
8           0  0  0    35  0  0   201   289   667     1192
__all__    16  0  0  1739  1  8  6664  2353  1033    11814


Overall Statistics:

Accuracy: 0.746995090571
95% CI: (0.7390532843935097, 0.75481502452165972)
No Information Rate: ToDo
P-Value [Acc > NIR]: 0.0
Kappa: 0.600293676006
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                          0  

In [15]:
df_features_weight.sort_values(by='importance',ascending=False)

,features,importance
4,SARKOZY_2012,0.072282
2,LE.PEN_2012,0.062837
3,HOLLANDE_2012,0.055898
1,POPULATION,0.048188
5,MELENCHON_2012,0.040285
38,education_SUP_ratio,0.033539
109,Nb.Actifs.Non.Salariés_ratio,0.029491
58,Dep.Moyenne.Salaires.Cadre.Horaires,0.026214
18,Q213,0.019468
70,Taux.Propriété,0.015806
